## Hierarchical classification using Local Classification per Parent Node technique
Here we use the same approach as in /solutions/custom_tokens/xlm_roberta_with_classification_start_span_token.ipynb to train the first classifier to predict the first class, and we train extra three classifiers for each of those classes that will provide us fine-grained classification.

THIS FILE REQUIRES 
 - fix in calculating accuracy in last test epoch (view the comment below)
 - hyperparameter optimization
 - (interventions to handle unbalanced data) - tried adding pos_weight to BCEWithLogitsLoss to second layer classifiers, made it worse
 - detailed score metrics for second layer classification, and explore errors because results are bad :/

In [12]:
from google.colab import drive
drive.mount('/content/drive')
sub1 = 'drive/My Drive/Colab Notebooks/subtask1.parquet'
print(sub1)

# from pathlib import Path
# wd = Path.cwd()
# wd = wd.parent.parent
# wd = wd / 'merged_data'
# sub1 = str(wd) + '/subtask1.parquet'
# print(sub1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/My Drive/Colab Notebooks/subtask1.parquet


In [13]:
import pandas as pd
df = pd.read_parquet(sub1)

In [14]:
df

,lang,art_name,entity,start,end,class1,classes2,text
0,BG,BG_670.txt,Запад,152,156,Antagonist,"[Conspirator, Instigator, Foreign Adversary]",Опитът на колективния Запад да „обезкърви Руси...
1,BG,BG_670.txt,САЩ,530,532,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
2,BG,BG_670.txt,НАТО,535,538,Antagonist,[Instigator],Опитът на колективния Запад да „обезкърви Руси...
3,BG,BG_670.txt,Украйна,578,584,Antagonist,[Foreign Adversary],Опитът на колективния Запад да „обезкърви Руси...
4,BG,BG_670.txt,украински войници,633,649,Innocent,[Victim],Опитът на колективния Запад да „обезкърви Руси...
...,...,...,...,...,...,...,...,...
2897,RU,RU-URW-1156.txt,Россией,559,565,Protagonist,[Peacemaker],Медведев: Даже в случае признания поражения Ки...
2898,RU,RU-URW-1156.txt,Москва,1005,1010,Protagonist,[Guardian],Медведев: Даже в случае признания поражения Ки...
2899,RU,RU-URW-1156.txt,НАТО,1298,1301,Antagonist,[Foreign Adversary],Медведев: Даже в случае признания поражения Ки...
2900,RU,RU-URW-1156.txt,Киевом,1948,1953,Antagonist,[Terrorist],Медведев: Даже в случае признания поражения Ки...


In [15]:
import re
def labelNum(row):
    if row['class1'] == 'Antagonist':
        return int(0)
    if row['class1'] == 'Innocent':
        return int(1)
    if row['class1'] == 'Protagonist':
        return int(2)
def cleanText(row):
    text = str(row['text'])
    #text = re.sub(r'[^\w\s]', ' ', text)
    text = text.replace('\n',' ').replace('  ', ' ')
    return text
df['label1'] = df.apply(labelNum,axis=1)
df['input'] = df.apply(cleanText,axis=1)

In [16]:
def labelNum2(row):
    labels2 = [0 for _ in range(12)]
    if row['label1'] == 2:
        #labels2 = [0 for _ in range(6)]
        if 'Guardian' in row['classes2']:
            labels2[0] = 1
        if 'Martyr' in row['classes2']:
            labels2[1] = 1
        if 'Peacemaker' in row['classes2']:
            labels2[2] = 1
        if 'Rebel' in row['classes2']:
            labels2[3] = 1
        if 'Underdog' in row['classes2']:
            labels2[4] = 1
        if 'Virtuous' in row['classes2']:
            labels2[5] = 1
    elif row['label1'] == 0:
        #labels2 = [0 for _ in range(12)]
        if 'Instigator' in row['classes2']:
           labels2[0] = 1
        if 'Conspirator' in row['classes2']:
            labels2[1] = 1
        if 'Tyrant' in row['classes2']:
            labels2[2] = 1
        if  'Foreign Adversary' in row['classes2']:
            labels2[3] = 1
        if 'Traitor' in row['classes2']:
            labels2[4] = 1
        if 'Spy' in row['classes2']:
            labels2[5] = 1
        if 'Saboteur' in row['classes2']:
            labels2[6] = 1
        if 'Corrupt' in row['classes2']:
            labels2[7] = 1
        if 'Incompetent' in row['classes2']:
            labels2[8] = 1
        if 'Terrorist' in row['classes2']:
            labels2[9] = 1
        if 'Deceiver' in row['classes2']:
            labels2[10] = 1
        if 'Bigot' in row['classes2']:
            labels2[11] = 1
    elif row['label1'] == 1:
        #labels2 = [0 for _ in range(4)]
        if 'Forgotten' in row['classes2']:
            labels2[0] = 1
        if 'Exploited' in row['classes2']:
            labels2[1] = 1
        if 'Victim' in row['classes2']:
            labels2[2] = 1
        if 'Scapegoat' in row['classes2']:
            labels2[3] = 1
    return labels2

df['label2'] = df.apply(labelNum2, axis=1)

In [17]:
def find_all_substring_start_end(text, substring):
    # Use re.finditer to find all occurrences of the substring in the text
    matches = re.finditer(re.escape(substring), text)

    # Collect the start and end indices of all matches
    positions = [(match.start(), match.end()) for match in matches]

    return positions
def adjust_start_end(row):
    org_text,cl_text,start,end,entity = str(row['text']),str(row['input']),int(row['start']),int(row['end']),str(row['entity'])
    ss1 = find_all_substring_start_end(org_text,entity)
    ss2 = find_all_substring_start_end(cl_text,entity)
    #print(ss1,ss2)
    #print(row['text'][start:end])
    a = 0
    for i in range(len(ss1)):
        if abs((ss1[i][0] - start) + (ss1[i][1] - end) ) <= 2:
            a = i
            break
    if org_text[ss1[a][0]:ss1[a][1]] != cl_text[ss2[a][0]:ss2[a][1]]:
        print("ERROR!")
    return ss2[a][0],ss2[a][1]
df['new_start_end'] = df.apply(adjust_start_end,axis=1)
print(df.loc[0])

lang                                                            BG
art_name                                                BG_670.txt
entity                                                       Запад
start                                                          152
end                                                            156
class1                                                  Antagonist
classes2              [Conspirator, Instigator, Foreign Adversary]
text             Опитът на колективния Запад да „обезкърви Руси...
label1                                                           0
input            Опитът на колективния Запад да „обезкърви Руси...
label2                        [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
new_start_end                                           (151, 156)
Name: 0, dtype: object


In [18]:
def addTokensToInput(row):
    inp = row['input']
    start,end = row['new_start_end']
    #print(start,end)
    start = int(start)
    end = int(end)
    token_input = inp[:start] + "[SPAN_START] " + inp[start:end] + " [SPAN_END]" + inp[end:]
    return token_input

df['span_input'] = df.apply(addTokensToInput,axis=1)

In [19]:
def upStartEnd(row):
    start,end = row['new_start_end']
    start += len("[SPAN_START] ")
    end += len("[SPAN_START] ")
    return start,end

df['new_start_end'] = df.apply(upStartEnd,axis = 1)

In [20]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3).to(device)
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def preprocess_function(examples):
    return tokenizer(examples['span_input'], padding=True, truncation=True,max_length=8192,return_offsets_mapping=True)

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [21]:
extraTokens = {
    "additional_special_tokens": ["[SPAN_START]", "[SPAN_END]"]
}
num_added_toks = tokenizer.add_special_tokens(extraTokens)
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250004, 768, padding_idx=1)

In [22]:
data = df.loc[ : , ['span_input', 'label1', 'label2', 'new_start_end', 'entity']]

In [23]:
data

,span_input,label1,label2,new_start_end,entity
0,Опитът на колективния Запад да „обезкърви Руси...,0,"[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","(164, 169)",Запад
1,Опитът на колективния Запад да „обезкърви Руси...,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(541, 544)",САЩ
2,Опитът на колективния Запад да „обезкърви Руси...,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(546, 550)",НАТО
3,Опитът на колективния Запад да „обезкърви Руси...,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","(589, 596)",Украйна
4,Опитът на колективния Запад да „обезкърви Руси...,1,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(644, 661)",украински войници
...,...,...,...,...,...
2897,Медведев: Даже в случае признания поражения Ки...,2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(569, 576)",Россией
2898,Медведев: Даже в случае признания поражения Ки...,2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(1015, 1021)",Москва
2899,Медведев: Даже в случае признания поражения Ки...,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","(1308, 1312)",НАТО
2900,Медведев: Даже в случае признания поражения [S...,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","(57, 63)",Киевом


In [24]:
data['tokenized']=data.apply(preprocess_function,axis=1)

In [25]:
def indexes(row):
    off_mask = row['tokenized']['offset_mapping']
    start,end = row['new_start_end'][0],row['new_start_end'][1]
    inds = list()
    for p in range(len(off_mask)):
        if off_mask[p][0] >= start and off_mask[p][1] <= end:
            if p != len(off_mask)-1:
                inds.append(p)
    #if len(inds) > 1:
        #print("GREATER THAN 1")
    if len(inds) == 0:
        print(start,end)
    return inds
data['indexes'] = data.apply(indexes,axis=1)

In [26]:
data['list'] = data['tokenized'].apply(lambda x: x['input_ids'])
data['attention'] = data['tokenized'].apply(lambda x: x['attention_mask'])
ids = data['list']
att = data['attention']
indexes = data['indexes']
tids = list()
tatt = list()
print(len(ids),len(att),len(indexes))
for i in range(len(ids)):
    tids.append(torch.tensor(ids[i]))
    tatt.append(torch.tensor(att[i]))

2902 2902 2902


In [27]:
sliced_ids = list()
sliced_ntids = list()
sliced_att = list()
key_inds = list()
key_ids = list()

def slices(index,size,context_size):
    if (size<context_size):
        return 0,size
    lower_c = int(context_size/2-1)
    upper_c = int(context_size/2)
    #print(lower_c,upper_c)
    if index < lower_c:
        return 0,context_size
    elif index >= lower_c:
        if index + upper_c > size:
            return index-(context_size-(size-index)), size
        else:
            return index-lower_c,index+upper_c+1


for i in range(len(tids)):
    slower,supper = slices(indexes[i][0],len(tids[i]),510)
    #key_tid = tids[i][indexes[i][0]]
    pid = ids[i][slower:supper]
    key_inds.append([])
    for j in indexes[i]:
        key_id = ids[i][j]
        if key_id not in pid:
           print(len(ids[i]),key_id,slower,supper,indexes[i])
        key_inds[i].append(pid.index(key_id))
    apid = tids[i][slower:supper]
    apatt = tatt[i][slower:supper]
    if 0 not in pid:
        apid = torch.cat((torch.tensor([0]),apid),dim=0)
        apatt = torch.cat((torch.tensor([1]),apatt),dim=0)
    if 2 not in pid:
        apid = torch.cat((apid,torch.tensor([2])),dim=0)
        apatt = torch.cat((apatt,torch.tensor([1])),dim=0)
    sliced_ids.append(apid)
    sliced_att.append(apatt)

Min = 10000
Max = 0
ind2 = 0
for i in range(len(indexes)):
    if len(sliced_ids[i]) < Min:
        Min = len(sliced_ids[i])
        ind2 = i

    if len(sliced_ids[i]) > Max:
        Max = len(sliced_ids[i])

In [28]:
input_ids = list()
att_mask = list()
for ten,att in zip(sliced_ids,sliced_att):
    if len(ten) < 512:
        padding_length = 512 - len(ten)
        padding_tensor = torch.full((padding_length,), tokenizer.pad_token_id, dtype=ten.dtype)
        padding_tensor2 = torch.full((padding_length,), 0, dtype=att.dtype)
        ten = torch.cat((ten,padding_tensor),dim=0)
        att = torch.cat((att,padding_tensor2),dim=0)
    input_ids.append(ten)
    att_mask.append(att)
inputIds = torch.stack(input_ids)
attMask = torch.stack(att_mask)

inputIds_np = inputIds.numpy()
attMask_np = attMask.numpy()
y1 = data['label1'].values
y2 = data['label2'].values

In [29]:
from sklearn.model_selection import train_test_split
X_train_ids, X_test_ids, X_train_mask, X_test_mask, y1_train, y1_test, y2_train, y2_test = train_test_split(
    inputIds_np, attMask_np, y1, y2, test_size=0.2, random_state=42, shuffle=True
)

In [30]:
import numpy as np
y2_train = np.array(y2_train.tolist(), dtype=np.int8)
y2_test = np.array(y2_test.tolist(), dtype=np.int8)

In [31]:
X_train_ids = torch.tensor(X_train_ids, dtype=torch.long).to(device)
X_test_ids = torch.tensor(X_test_ids, dtype=torch.long).to(device)
X_train_mask = torch.tensor(X_train_mask, dtype=torch.long).to(device)
X_test_mask = torch.tensor(X_test_mask, dtype=torch.long).to(device)
y1_train = torch.tensor(y1_train, dtype=torch.long).to(device)
y1_test = torch.tensor(y1_test, dtype=torch.long).to(device)
y2_train = torch.tensor(y2_train, dtype=torch.long).to(device)
y2_test = torch.tensor(y2_test, dtype=torch.long).to(device)

In [32]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train_ids, X_train_mask, y1_train, y2_train)
test_dataset = TensorDataset(X_test_ids, X_test_mask, y1_test, y2_test )

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True) #shuffle=True provides data shuffle for batches in different epochs
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [33]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn

#first layer classifier
classifier = nn.Linear(model.config.hidden_size, 3).to(device)
optimizer = AdamW(model.parameters(), lr=8e-6)
criterion = nn.CrossEntropyLoss()

In [34]:
# not used - will be tried out
# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2., alpha=0.25, num_classes=3):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.alpha = alpha
#         self.num_classes = num_classes
#         self.cross_entropy_loss = nn.CrossEntropyLoss(reduction='none')

#     def forward(self, inputs, targets):
#         ce_loss = self.cross_entropy_loss(inputs, targets)
#         p_t = torch.exp(-ce_loss)  # Probability of correct class
#         focal_loss = self.alpha * (1 - p_t) ** self.gamma * ce_loss
#         return focal_loss.mean()

#criterion = FocalLoss(gamma=2., alpha=0.25)

In [35]:
# second layer classifier model
# class SecondLayerClassifier(nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(SecondLayerClassifier, self).__init__()
#         self.fc = nn.Linear(input_dim, num_classes)

#     def forward(self, x):
#         return torch.sigmoid(self.fc(x))

criterion2 = nn.BCEWithLogitsLoss()

class SecondLayerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes):
        """
        Multi-layer classifier for second-layer classification.

        Args:
            input_dim (int): Dimension of the input features.
            hidden_dims (list of int): List of hidden layer dimensions.
            num_classes (int): Number of output classes.
        """
        super(SecondLayerClassifier, self).__init__()

        self.num_classes = num_classes

        layers = []
        current_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(current_dim, hidden_dim))
            layers.append(nn.GELU())
            current_dim = hidden_dim

        layers.append(nn.Linear(current_dim, num_classes))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [36]:
#second layer classifiers
hidden_dimension = [768, 768, 768, 768, 768, 768]

child_classifiers = {
    int(0) : SecondLayerClassifier(input_dim=model.config.hidden_size, hidden_dims=hidden_dimension, num_classes=12).to(device) , #Antagonist
    int(1) : SecondLayerClassifier(input_dim=model.config.hidden_size, hidden_dims=hidden_dimension, num_classes=4).to(device) , #Innocent
    int(2) : SecondLayerClassifier(input_dim=model.config.hidden_size, hidden_dims=hidden_dimension,  num_classes=6).to(device) , #Protagonist
}

second_layer_optimizers = {
    name: AdamW(sec_layer_classifier.parameters(), lr=0.0001)
    for name, sec_layer_classifier in child_classifiers.items()
}

In [38]:
import numpy as np
# for the confusion matrix in the end
all_preds = np.array([], dtype=np.int8)
all_labels = np.array([], dtype=np.int8)

In [39]:
num_epochs = 6
debug = 0
for epoch in range(num_epochs):

    model.train()
    classifier.train()
    child_classifiers[0].train()
    child_classifiers[1].train()
    child_classifiers[2].train()

    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    second_layer_stats = {
        cls: {'loss': 0.0, 'correct': 0, 'total': 0}
        for cls in child_classifiers.keys()
    }

    train_progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{num_epochs}")

    for batch in train_progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels_1 = batch[2].to(device)
        labels_2 = batch[3].to(device)  # second-layer labels

        batch_size = input_ids.size(0)

        #taking the output from BERT model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_1, output_hidden_states=True)
        hidden_states = outputs.hidden_states[-1]

        span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
        start_mask = (input_ids == span_start_token_id)
        entity_representations = []
        start_indices = start_mask.nonzero(as_tuple=True)[1]
        valid_spans = (start_indices != -1) 
        valid_start_indices = start_indices[valid_spans]

        for i in range(batch_size):
            entity_tokens = hidden_states[i, valid_start_indices[i]] #for this version only start span tokens
            entity_representations.append(entity_tokens)

        entity_representations = torch.stack(entity_representations, dim=0)

        #first layer classification
        logits = classifier(entity_representations)
        loss = criterion(logits, labels_1)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=-1)
        correct_predictions += (preds == labels_1).sum().item()
        total_predictions += labels_1.size(0)

        train_progress_bar.set_postfix({'loss': loss.item()})

        #second layer classification
        for cls, second_layer_classifier in child_classifiers.items():
            second_layer_indices = (labels_1 == cls).nonzero(as_tuple=True)[0]

            if second_layer_indices.size(0) > 0:
                second_layer_inputs = entity_representations[second_layer_indices].detach()

                num_classes = second_layer_classifier.num_classes
                second_layer_labels = labels_2[second_layer_indices, :num_classes].float()

                child_logits = second_layer_classifier(second_layer_inputs)
                child_loss = criterion2(child_logits, second_layer_labels)

                second_layer_optimizers[cls].zero_grad()
                child_loss.backward()
                second_layer_optimizers[cls].step()

                #calculating accuracy
                second_layer_stats[cls]['loss'] += child_loss.item() * second_layer_labels.size(0)
                child_preds = (torch.sigmoid(child_logits) > 0.35).int()
                correct = ((child_preds == second_layer_labels.int()).all(dim=1)).sum().item() #strict accuracy
                second_layer_stats[cls]['correct'] += correct
                second_layer_stats[cls]['total'] += second_layer_labels.size(0)

    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_predictions

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training loss: {avg_train_loss:.4f}, Training accuracy: {train_accuracy:.4f}")

    for cls, stats in second_layer_stats.items():
        if stats['total'] > 0:
            avg_loss = stats['loss'] / stats['total']
            accuracy = stats['correct'] / stats['total']
        else:
            avg_loss = 0.0
            accuracy = 0.0
        print(f"Second-Layer Classifier: {cls}, Avg Loss: {avg_loss:.4f}, (strict) Accuracy: {accuracy:.4f}, Total: {stats['total']}")

    model.eval()
    classifier.eval()
    child_classifiers[0].eval()
    child_classifiers[1].eval()
    child_classifiers[2].eval()

    test_loss = 0
    correct_test_predictions = 0
    total_test_predictions = 0

    test_stats_per_classifier = {
        cls: {'loss': 0.0, 'correct': 0, 'total': 0}
        for cls in child_classifiers.keys()
    }
    overall_stats = {'loss': 0.0, 'correct': 0, 'total': 0}

    test_progress_bar = tqdm(test_dataloader, desc=f"Test Epoch {epoch + 1}/{num_epochs}")

    with torch.no_grad():
        for batch in test_progress_bar:

            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels_1 = batch[2].to(device)
            labels_2 = batch[3].to(device)

            batch_size = input_ids.size(0)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels_1, output_hidden_states=True)

            hidden_states = outputs.hidden_states[-1]

            span_start_token_id = tokenizer.convert_tokens_to_ids('[SPAN_START]')
            start_mask = (input_ids == span_start_token_id)
            entity_representations = []
            start_indices = start_mask.nonzero(as_tuple=True)[1]
            valid_spans = (start_indices != -1)
            valid_start_indices = start_indices[valid_spans]

            # extract entity tokens for every sample in batch
            for i in range(batch_size):
                entity_tokens = hidden_states[i, valid_start_indices[i]]
                entity_representations.append(entity_tokens)

            entity_representations = torch.stack(entity_representations, dim=0)

            logits = classifier(entity_representations)
            loss = criterion(logits, labels_1)
            test_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)

            #if epoch is the last epoch we want to redirect data to second layer classifier according to predicted label
            if epoch == num_epochs-1:

                #second layer classification
                for cls, second_layer_classifier in child_classifiers.items():
                    second_layer_indices = (preds == cls).nonzero(as_tuple=True)[0] #notice here we use "preds" instead of "labels_1" which was used in training

                    if second_layer_indices.size(0) > 0:
                        second_layer_inputs = entity_representations[second_layer_indices]

                        num_classes = second_layer_classifier.num_classes
                        second_layer_labels = labels_2[second_layer_indices, :num_classes].float()

                        child_logits = second_layer_classifier(second_layer_inputs)
                        child_loss = criterion2(child_logits, second_layer_labels)

                        test_stats_per_classifier[cls]['loss'] += child_loss.item() * second_layer_labels.size(0) #is it necessary to multiply it with batch size?

                        child_preds = (torch.sigmoid(child_logits) > 0.35).int()

                        correct = ((child_preds == second_layer_labels.int()).all(dim=1)).sum().item() #strict accuracy
                        test_stats_per_classifier[cls]['correct'] += correct
                        test_stats_per_classifier[cls]['total'] += second_layer_labels.size(0) 

                        overall_stats['loss'] += child_loss.item() * second_layer_labels.size(0)
                        overall_stats['correct'] += correct #TODO ISPRAVITI OVO, ako je krivi preds onda ne moze biti tocna klasifikacija druge razine
                        overall_stats['total'] += second_layer_labels.size(0)

                #for confusion matrix in determing first class
                all_preds = np.concatenate((all_preds, preds.cpu().numpy()))
                all_labels = np.concatenate((all_labels, labels_1.cpu().numpy()))

            correct_test_predictions += (preds == labels_1).sum().item()
            total_test_predictions += labels_1.size(0)

            test_progress_bar.set_postfix({'loss': loss.item()})

    avg_test_loss = test_loss / len(test_dataloader)
    test_accuracy = correct_test_predictions / total_test_predictions

    print(f"Test loss: {avg_test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

    if epoch == num_epochs-1:

        for cls, stats in test_stats_per_classifier.items():
            if stats['total'] > 0:
                avg_loss = stats['loss'] / stats['total']
                accuracy = stats['correct'] / stats['total']
            else:
                avg_loss = 0.0
                accuracy = 0.0
            print(f"Second-Layer Classifier: {cls}, Avg Loss: {avg_loss:.4f}, (strict) Accuracy: {accuracy:.4f}, Total: {stats['total']}")

        overall_accuracy = overall_stats['correct'] / overall_stats['total'] if overall_stats['total'] > 0 else 0.0
        print(f"OVERALL STRICT ACCURACY AFTER THE SECOND LAYER CLASSIFICATION: {overall_accuracy:.4f}")
        #strict accuracy means all the labels are correctly predicted for the input

Training Epoch 1/6: 100%|██████████| 146/146 [03:43<00:00,  1.53s/it, loss=1.35]


Epoch 1/6
Training loss: 0.9450, Training accuracy: 0.5579
Second-Layer Classifier: 0, Avg Loss: 0.3438, (strict) Accuracy: 0.0541, Total: 1145
Second-Layer Classifier: 1, Avg Loss: 0.3483, (strict) Accuracy: 0.6929, Total: 508
Second-Layer Classifier: 2, Avg Loss: 0.4481, (strict) Accuracy: 0.2275, Total: 668


Test Epoch 1/6: 100%|██████████| 37/37 [00:17<00:00,  2.16it/s, loss=0.591]


Test loss: 0.7269, Test accuracy: 0.6885


Training Epoch 2/6: 100%|██████████| 146/146 [03:56<00:00,  1.62s/it, loss=0.499]


...

Test loss: 0.4846, Test accuracy: 0.8227


Training Epoch 6/6: 100%|██████████| 146/146 [03:56<00:00,  1.62s/it, loss=0.0257]


Epoch 6/6
Training loss: 0.2308, Training accuracy: 0.9164
Second-Layer Classifier: 0, Avg Loss: 0.2546, (strict) Accuracy: 0.1852, Total: 1145
Second-Layer Classifier: 1, Avg Loss: 0.2167, (strict) Accuracy: 0.8110, Total: 508
Second-Layer Classifier: 2, Avg Loss: 0.3686, (strict) Accuracy: 0.2096, Total: 668


Test Epoch 6/6: 100%|██████████| 37/37 [00:17<00:00,  2.10it/s, loss=0.281]

Test loss: 0.4934, Test accuracy: 0.8503
Second-Layer Classifier: 0, Avg Loss: 0.2748, (strict) Accuracy: 0.1521, Total: 309
Second-Layer Classifier: 1, Avg Loss: 0.2783, (strict) Accuracy: 0.8163, Total: 98
Second-Layer Classifier: 2, Avg Loss: 0.3787, (strict) Accuracy: 0.2126, Total: 174
OVERALL STRICT ACCURACY AFTER THE SECOND LAYER CLASSIFICATION: 0.2823


In [40]:
# TODO for hierarchical classification
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

# cm = confusion_matrix(all_labels, all_preds, labels=[0, 1, 2])

# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Antagonist', 'Innocent', 'Protagonist'], yticklabels=['Antagonist', 'Innocent', 'Protagonist'])
# plt.xlabel('Predicted Labels')
# plt.ylabel('True Labels')
# plt.title('Confusion Matrix')
# plt.show()